In [1]:
# Use this initial code to work in the notebook as if it were a module, that 
# is, to be able to export classes and functions from other subpackages.

import pyprojroot
import sys

package_path = pyprojroot.here().__str__()
if package_path not in sys.path:
    sys.path.append(package_path)

In [2]:
from utils import AbsPaths
import os
import pandas as pd
import numpy as np
import glob
import re

In [4]:
def get_list_files(start_file_name = "atp_matches", path = None):
    if path is None:
        path = AbsPaths().get_abs_path_folder(folder_name="raw")

    return sorted(glob.glob(path + f"{start_file_name}" + "*.csv"))



In [8]:
def validate_year(list_files, year, index):
    if index not in [0,-1]:
        raise IndexError("You need provide 0 (start) or -1 (end)")
    if year is None:
        file = list_files[index]
        year = "/".join(re.findall(r"\d+",file.split(os.sep)[-1]))

    else:
        file = [f for f in list_files if year in f]

        if len(file) == 0:
            raise FileNotFoundError("You need provide an available start year")
        
        else:
            file = file[0]

    return file, year

In [51]:
def join_tables(start_year:str = None, start_file_name = "atp_matches", end_year = None, path_read = None, path_to_save = None):
    if path_read is None:
        path_read = AbsPaths().get_abs_path_folder(folder_name="raw")
    list_files = get_list_files(start_file_name = start_file_name, path = path_read)

    start_file, start_year = validate_year(list_files=list_files, year=start_year, index = 0)
    end_year = validate_year(list_files=list_files, year=end_year, index = -1)[1]

    data = pd.read_csv(start_file,sep=",")

    for file in list_files:
        if file == start_file:
            pass
        year_file = "/".join(re.findall(r"\d+",file.split(os.sep)[-1]))
        # print(start_year, end_year, year_file)
        if (year_file > start_year) and (year_file <= end_year):
            df = pd.read_csv(file, sep=',')
            data = pd.concat([data, df],ignore_index=True)
        
        data = data[data["tourney_level"] == "G"]
        
    if path_to_save is None:
        path_to_save = AbsPaths().get_abs_path_folder(folder_name="interim")
    
    data.to_csv(path_to_save + f"{start_file_name}_historic.csv", index=False)

    # return data

    

In [52]:
join_tables()